In [ ]:
## Import packages
import os
import csv
import random
import math
import ntpath
import numpy as np
import pandas as pd
import time
from pathlib import Path
from scipy.io import wavfile
from tqdm import tqdm

import tensorflow as tf
from tensorflow import gfile
from tensorflow import logging

In [ ]:
serval_data_folder = "../data"

In [ ]:
input_wav_samples_folder                     = serval_data_folder + "/wav_samples_custom"
wav_samples_custom_labels_csv                = input_wav_samples_folder + "/wav_samples_custom_labels.csv"
target_wav_samples_custom_all_enumerated_csv = serval_data_folder + "/wav_samples_custom/wav_samples_custom_all_enumerated_and_labeled.csv"

In [ ]:
# read class labels
df_class_labels = pd.read_csv(wav_samples_custom_labels_csv, sep=";")
display(df_class_labels)

In [ ]:
# Enumerate all the available wav files, label them and store a data-frame of the result
def collectAndLabelSamples(full_path_name, label, mid, source, display_name):
    ## Read and rewrite all test files
    files = gfile.Glob(str(full_path_name + "/*.wav"))
    
    df = pd.DataFrame(columns=['label', 'mid', 'display_name', 'source', 'filename', 'filepath'])
    for file in tqdm(files):
        df_row = pd.DataFrame({'label':[label], 
                               'mid':[mid], 
                               'display_name':[display_name], 
                               'source':source, 
                               'filename':[ntpath.basename(file)],
                               'filepath':[file]})
        df = df.append(df_row, ignore_index = True)
    return(df)

In [ ]:
## Enumerate all custom wav-files and label them
df_all_wav_samples = pd.DataFrame(columns=['label', 'mid', 'display_name', 'source', 'filename', 'filepath'])
for index, row in df_class_labels.iterrows():
    if row['enabled'] == 1:
        print('Processing record ', index, '; label=', row.label, ', description=', row.description, ', path=', row.folder_name, sep='')
        df_row = collectAndLabelSamples(input_wav_samples_folder + '/' + row.folder_name, 
                                        row.label,
                                        row.mid,
                                        row.source,
                                        row.description)
        df_all_wav_samples = df_all_wav_samples.append(df_row, ignore_index = True)
    else:
        print('Skipping record ', index, '; label=', row.label, ', description=', row.description, ', path=', row.folder_name, sep='')

In [ ]:
## Write df_all_wav_samples to csv
df_all_wav_samples.to_csv(target_wav_samples_custom_all_enumerated_csv, sep=';')

In [ ]:
display(df_all_wav_samples.head())
display(df_all_wav_samples.groupby(['label', 'mid', 'display_name']).agg(['nunique']))
display(df_all_wav_samples.agg(['nunique']))